In [ ]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
from tkinter import Tk, Label, Button, filedialog, Canvas, Entry, Frame, Scrollbar, Listbox, Checkbutton, IntVar, messagebox
from PIL import Image, ImageTk

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Define constants
IMG_SIZE = 64
NUM_CLASSES = 15

# Define paths to CSV files
TRAIN_CSV_PATH = 'Training_set.csv'
TEST_CSV_PATH = 'Testing_set.csv'

# Define the base directory where images are stored
IMAGE_BASE_DIR = r'C:\Users\Pragathi.M.Shetty\OneDrive\Desktop\archive\Human Action Recognition\train'

# Define a mapping of class indices to action names
ACTION_NAMES = [
    "sitting", "sleeping", "clapping", "dancing", "drinking",
    "eating", "fighting", "hugging", "laughing", "listening_to_music",
    "running", "texting", "using_laptop", "calling", "other"
]

def load_and_preprocess_images_from_csv(csv_path, has_labels=True):
    data = []
    labels = []
    df = pd.read_csv(csv_path)
    for _, row in df.iterrows():
        img_path = os.path.join(IMAGE_BASE_DIR, row['image_path'])
        if has_labels:
            label = row['label']
        if os.path.exists(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                data.append(img)
                if has_labels:
                    labels.append(label)
            else:
                print(f"Warning: Could not read image {img_path}")
        else:
            print(f"Warning: Image path does not exist: {img_path}")
    data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    data = data / 255.0  # Normalize the images
    labels = np.array(labels) if has_labels else None
    return data, labels

# Load and preprocess the dataset
X_train, y_train = load_and_preprocess_images_from_csv(TRAIN_CSV_PATH)
X_test, _ = load_and_preprocess_images_from_csv(TEST_CSV_PATH, has_labels=False)

# Update NUM_CLASSES based on unique labels
unique_labels = np.unique(y_train)
NUM_CLASSES = len(unique_labels)
print(f"Unique labels: {unique_labels}")
print(f"Number of classes: {NUM_CLASSES}")

# Convert labels to categorical format
y_train = pd.Categorical(y_train).codes  # Convert string labels to numeric codes
y_train = to_categorical(y_train, NUM_CLASSES)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_acc}')

# Plot training & validation accuracy values
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

plot_training_history(history)

# Function to predict action on a new image
def predict_action(image_path, model):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 1) / 255.0
    prediction = model.predict(img)
    action_class = np.argmax(prediction)
    action_prob = prediction[0][action_class]
    return action_class, action_prob

# Simple GUI with Tkinter
class ActionRecognitionGUI:
    def __init__(self, master):
        self.master = master
        master.title("Action Recognition System")
        
        # Create frames
        self.top_frame = Frame(master)
        self.top_frame.pack(side='top', fill='x')
        
        self.middle_frame = Frame(master)
        self.middle_frame.pack(fill='x')
        
        self.bottom_frame = Frame(master)
        self.bottom_frame.pack(side='bottom', fill='x')
        
        # Top frame widgets
        self.label = Label(self.top_frame, text="Select an image to predict action.")
        self.label.pack()
        
        self.predict_button = Button(self.top_frame, text="Select Image", command=self.load_image)
        self.predict_button.pack()
        
        self.save_button = Button(self.top_frame, text="Save Model", command=self.save_model)
        self.save_button.pack()
        
        self.load_button = Button(self.top_frame, text="Load Model", command=self.load_model)
        self.load_button.pack()
        
        self.reset_button = Button(self.top_frame, text="Reset", command=self.reset)
        self.reset_button.pack()
        
        # Middle frame widgets
        self.image_label = Label(self.middle_frame)
        self.image_label.pack()
        
        self.result_label = Label(self.middle_frame, text="", font=("Helvetica", 16))
        self.result_label.pack()
        
        # Bottom frame widgets
        self.canvas = Canvas(self.bottom_frame, width=300, height=300)
        self.canvas.pack()
        
        self.entry = Entry(self.bottom_frame)
        self.entry.pack()
        
        self.scrollbar = Scrollbar(self.bottom_frame)
        self.scrollbar.pack(side='right', fill='y')
        
        self.listbox = Listbox(self.bottom_frame, yscrollcommand=self.scrollbar.set)
        self.listbox.pack(side='left', fill='both', expand=True)
        self.scrollbar.config(command=self.listbox.yview)
        
        # Add example actions to listbox
        self.update_listbox()
        
        # Checkbox for additional options
        self.var = IntVar()
        self.checkbox = Checkbutton(master, text="Enable advanced options", variable=self.var)
        self.checkbox.pack()

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            img = Image.open(file_path)
            img = img.resize((250, 250), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(img)
            self.image_label.config(image=img)
            self.image_label.image = img
            
            predicted_class, prediction_prob = predict_action(file_path, model)
            action_name = self.get_action_name(predicted_class)
            self.result_label.config(text=f"Predicted Action: {action_name}\nProbability: {prediction_prob:.2f}")

    def update_listbox(self):
        # Populate listbox with action names
        self.listbox.delete(0, 'end')  # Clear existing entries
        for action in ACTION_NAMES:
            self.listbox.insert('end', action)

    def get_action_name(self, class_id):
        # Return action name based on the class index
        return ACTION_NAMES[class_id] if class_id < len(ACTION_NAMES) else "Unknown"

    def save_model(self):
        model.save('action_recognition_model.h5')
        messagebox.showinfo("Save Model", "Model saved successfully.")

    def load_model(self):
        global model
        model = load_model('action_recognition_model.h5')
        messagebox.showinfo("Load Model", "Model loaded successfully.")

    def reset(self):
        self.result_label.config(text="")
        self.image_label.config(image='')
        self.image_label.image = None
        self.entry.delete(0, 'end')

if __name__ == "__main__":
    root = Tk()
    app = ActionRecognitionGUI(root)
    root.mainloop()
